<a href="https://colab.research.google.com/github/swrogers/tensorflow-udemy/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow.

More specifically, we'll cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors and NumPy
* Using @tf.function (speeding things up)
* Using GPU/TPUs with Tensorflow
* Exercises

## Introduction to Tensors

In [3]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.4.1


In [4]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [5]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [6]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [7]:
# Check the dimension of the vector
vector.ndim

1

In [8]:
# Create a matrix (more than 1 dimension)
matrix = tf.constant([[10, 7],
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [9]:
matrix.ndim

2

In [10]:
# Create another matrix
another_matrix = tf.constant([[10.,7.],
                              [3.,2.],
                              [8.,9.]],
                             dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [11]:
# What is the ndim of this?
another_matrix.ndim

2

In [12]:
# The total number of dimensions is the number of elements in the shape..

In [13]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                       [[7, 8, 9],
                        [10, 11, 12]],
                       [[13, 14, 15],
                        [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [14]:
tensor.ndim

3

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (where n can be any number, a 0-dim is a scalar, a 1-dim is a vector)

### Creating tensors with `tf.Variable`


In [15]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [16]:
# Let's try to change one of the elements in the changeable tensor
# changeable_tensor[0] = 7

In [17]:
# How about try .assign()
changeable_tensor[0].assign(7)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [18]:
# Let's try the unchangeable tensor...
# unchangeable_tensor[0].assign(7)
unchangeable_tensor

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>

Note: Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

Random tensors are tensors of some arbitrary size containing random numbers.

In [19]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))
random_1
random_2

# Are the equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor


In [20]:
# Shuffle a tensor (valuable for when you want to shuffle data so the inherent order does not effect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2,5]])
not_shuffled.ndim

2

In [21]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [22]:
# Shuffle the non shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

In [23]:
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

**Exercise:** Read through TensorFlow documentation on random seed generation

It looks like if we want our shuffled tensors to be in the same order, we use the global level as well as the operation level random seed.

In [24]:
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]], dtype=int32)>

### Other ways to make tensors

In [25]:
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [26]:
# Create a tensor of all zeroes
tf.zeros([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on GPUs.

In [27]:
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32)
B = tf.constant(numpy_A)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [28]:
A = tf.constant(numpy_A, shape=(2,3,4)) # numpy_A has 24 elements, so the shape of the tensor needs to hold equal amount: 2*3*4 = 24
C = tf.constant(numpy_A, shape=(3,8))   # ... 3*8 = 24
A, B, C

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>)

### Getting information from tensors

When dealing with tensors - be aware of the following attributes
* Shape
* Rank
* Axis or dimension
* Size

In [29]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros([2, 3, 4, 5]) # 2 large groups, each of 3, with 4 arrays of length 5
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [30]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [31]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in tensor:", tf.size(rank_4_tensor).numpy())
print("Total number of elements in tensor:", tf.size(rank_4_tensor))

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in tensor: 120
Total number of elements in tensor: tf.Tensor(120, shape=(), dtype=int32)


### Indexing Tensors

Tensors can be indexed just like python lists.

In [32]:
some_list = [1,2,3,4]
some_list[:2]

[1, 2]

In [33]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [34]:
# Get the first element from each dimension from each index except for the final one
some_list[:1]

[1]

In [35]:
rank_4_tensor[:1, :1, :1]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [36]:
# or...
rank_4_tensor[:1,:1,:1,:]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [37]:
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [38]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[10, 7],
                             [3,4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [39]:
# Get the last item of each rank 2 tensor
rank_2_tensor[:,-1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [40]:
# Add in extra dimension to rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [41]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # -1 means expand final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [42]:
tf.expand_dims(rank_2_tensor, axis=0) # put extra on the front

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

In [43]:
rank_2_tensor


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

### Manipulating tensors (tensor operations)

**Basic operations**
`+`, `-`, `*`, `/`

In [44]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3,4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [45]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [46]:
# Multiplication
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [47]:
# Subtraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [48]:
# We can use the tensorflow built-in function too - the tensorflow versions are sped up/optimal
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [49]:
# The orinal tensor remains unchanged:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

**Matrix multiplication**

In machine learning, matrix multiplication is one of the most common operations

There are two rules that tensors (or matrices) need to fullfil:
1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [50]:
# Matrix multiplication in tensorflow (dot product)
tf.matmul(tensor, tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [51]:
tA = tf.constant([[1, 2, 5],
                  [7, 2, 1],
                  [3, 3, 3]])
tB = tf.constant([[3, 5],
                  [6, 7],
                  [1, 8]])
tf.matmul(tA, tB)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [52]:
# Matrix multiplication with Python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [53]:
# Create a tensor of 3, 2
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
# and another
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [54]:
# Try to matrix multiply tensors of same shape
# X @ Y

In [55]:
# tf.matmul(X,Y)

In [56]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [57]:
# Try to multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [58]:
# With tf.matmul...
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [59]:
# ...changing the shape of X instead
tf.matmul(tf.reshape(X, shape=(2,3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [60]:
# Can do the same with transpose, sort of
tf.transpose(X), tf.reshape(X, shape=(2,3)), X

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>)

In [61]:
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`

In [62]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [63]:
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [64]:
# Perform matrix mult between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [65]:
# Perform matrix mult between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [66]:
# Check the values of Y, reshape Y and transposed Y
print("Normal Y: ")
print(Y, "\n")
print("Y reshaped to (2,3): ")
print(tf.reshape(Y, shape=(2,3)), "\n")
print("Y transposed: ")
print(tf.transpose(Y))

Normal Y: 
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2,3): 
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed: 
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


In [67]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

Generally, when multiplying tensors, transpose one to satisfy matrix multiplication rules

### Changing the datatype of a tensor


In [68]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [69]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [70]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [71]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

### Aggregating Tensors

_Aggregating tensors_: Condensing them from multiple values to a smaller amount of values.

In [72]:
# Get the absolute values
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [73]:
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregration:
* Minimum
* Maximum
* Mean
* Sum


In [74]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))

In [75]:
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([92, 21, 23, 15, 15, 24, 17, 43, 13, 11, 88, 85, 49, 51, 52, 57, 38,
       46, 48,  4, 59, 10, 96, 45, 75, 78, 34,  0, 90,  5, 11, 47, 51,  4,
       51, 77, 85, 20, 51,  1, 85, 81, 43, 14, 85, 23, 99, 76, 34, 78])>

In [76]:
E.ndim, E.shape, tf.size(E)

(1, TensorShape([50]), <tf.Tensor: shape=(), dtype=int32, numpy=50>)

In [77]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [78]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=99>

In [79]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=46>

In [80]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2300>

**Exercise** Find the variance and standard deviation of `E`.

In [81]:
# Standard Deviation (reduce_std requires a real/complex input, thus the cast)
tf.math.reduce_std(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=29.9366>

In [82]:
# Variance (lives in tensorflow_probability)
import tensorflow_probability as tfp
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int64, numpy=896>

In [83]:
# Another variance
tf.math.reduce_variance(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=896.2>

### Find the positional minimum and maximum of a tensor



In [84]:
# Create a new tensor for finding position min and max
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [85]:
# Find the positional maximum - the index of the max entry
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [86]:
# Index on our largest value position - pull the value of the max
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [87]:
# Find the max value of F - another method
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [88]:
# Check for equality - these should be equal
assert F[tf.argmax(F)] == tf.reduce_max(F)
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [89]:
# Find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [90]:
# Find the minimum value using the positional minimum
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

In [91]:
# Find the minimum using reduce_min
tf.reduce_min(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (removing all single dimensions)

In [92]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [93]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [94]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>, TensorShape([50]))

### One-hot encoding tensors
A form of numerical encoding

In [95]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, cyan
# one hot encode the list
tf.one_hot(some_list, depth=4) # depth is required

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [96]:
# Specify custom values for on-hot encoding (rarely used)
tf.one_hot(some_list, depth=4, on_value="on", off_value="off")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'on', b'off', b'off', b'off'],
       [b'off', b'on', b'off', b'off'],
       [b'off', b'off', b'on', b'off'],
       [b'off', b'off', b'off', b'on']], dtype=object)>

### Squaring, log, square root

In [97]:
# Create a new tensor
H = tf.range(1,10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [98]:
# Square
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [99]:
# Square root (requires non int type)
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [100]:
# Log (requires non int type)
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy
TensorFlow interacts with NumPy arrays

**NOTE** One of the main differences between a TensorFlow tensor and a NumPy array is that a TF Tensor can be run on a GPU/TPU for faster processing.

In [101]:
# Create a tensor directly from a numpy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [102]:
# Convert a tensor back into a numpy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [103]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [104]:
J = tf.constant([3.])
J.numpy(), J.numpy()[0]

(array([3.], dtype=float32), 3.0)

In [105]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPUs
Runtime -> Change Runtime Type

In [106]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [107]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [108]:
!nvidia-smi

Fri Apr 30 17:43:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    30W /  70W |    224MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Note:** If you have access to a cuda-enabled gpu, tensorflow will automatically use it when possible

# Exercises

## Create a vector, scalar, matrix and tensor with values of your choosing using tf.constant().

In [109]:
# Create a scalar (0 dimensional)
my_scalar = tf.constant(19)
my_scalar, my_scalar.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=19>, 0)

In [110]:
# Create a vector (1 dimensional)
my_vector = tf.constant([0, 12, 32])
my_vector, my_vector.ndim

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 0, 12, 32], dtype=int32)>,
 1)

In [111]:
# Create a matrix (2 dimensional)
my_matrix = tf.constant([[0, 1, 2],
                         [3, 4, 5]])
my_matrix, my_matrix.ndim

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[0, 1, 2],
        [3, 4, 5]], dtype=int32)>, 2)

In [112]:
# Create a tensor (n dimensional)
my_tensor = tf.constant([[[0, 1, 2],
                          [3, 4, 5],
                          [6, 7, 8]],
                         [[4, 5, 6],  
                          [7, 8, 9],
                          [10, 11, 12]]])
my_tensor, my_tensor.ndim

(<tf.Tensor: shape=(2, 3, 3), dtype=int32, numpy=
 array([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],
 
        [[ 4,  5,  6],
         [ 7,  8,  9],
         [10, 11, 12]]], dtype=int32)>, 3)

## Find the shape, rank and size of each of the above

In [113]:
# Find the shape
my_scalar.shape, my_vector.shape, my_matrix.shape, my_tensor.shape

(TensorShape([]),
 TensorShape([3]),
 TensorShape([2, 3]),
 TensorShape([2, 3, 3]))

In [114]:
# Find the rank
my_scalar.ndim, my_vector.ndim, my_matrix.ndim, my_tensor.ndim

(0, 1, 2, 3)

In [115]:
# Find the size
tf.size(my_scalar), tf.size(my_vector), tf.size(my_matrix), tf.size(my_tensor)

(<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=3>,
 <tf.Tensor: shape=(), dtype=int32, numpy=6>,
 <tf.Tensor: shape=(), dtype=int32, numpy=18>)

## Create two tensors containing random values between 0 and 1 with shape [5, 300]

In [116]:
tf.random.set_seed(42)
random_A = tf.constant(tf.random.uniform(shape=(5,300)))
random_B = tf.constant(tf.random.uniform(shape=(5,300)))
random_A, random_A.shape, random_B, random_B.shape

(<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.6645621 , 0.44100678, 0.3528825 , ..., 0.31410468, 0.7593535 ,
         0.03699052],
        [0.532024  , 0.29129946, 0.10571766, ..., 0.54052293, 0.31425726,
         0.2200619 ],
        [0.08404207, 0.03614604, 0.97732127, ..., 0.21516645, 0.9786098 ,
         0.00726748],
        [0.7396945 , 0.6653172 , 0.0787828 , ..., 0.7117733 , 0.07013571,
         0.9409125 ],
        [0.15861344, 0.12024033, 0.27218235, ..., 0.8824879 , 0.1432488 ,
         0.44135118]], dtype=float32)>,
 TensorShape([5, 300]),
 <tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.68789124, 0.48447883, 0.9309944 , ..., 0.6920762 , 0.33180213,
         0.9212563 ],
        [0.27369928, 0.10631859, 0.6218617 , ..., 0.4382149 , 0.30427706,
         0.51477313],
        [0.00920248, 0.37280262, 0.8177401 , ..., 0.56786287, 0.49201214,
         0.9892651 ],
        [0.88608265, 0.08672249, 0.12160683, ..., 0.91770685, 0.72545695,
         0.828

## Multiply those two tensors using matrix multiplication

In [117]:
tf.matmul(random_A, tf.transpose(random_B))

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[80.333435, 73.40498 , 77.15961 , 73.98368 , 80.90053 ],
       [75.14637 , 68.80438 , 74.24303 , 71.84184 , 75.60205 ],
       [79.7594  , 75.64456 , 77.79758 , 74.748726, 80.55984 ],
       [75.085266, 69.06408 , 74.30776 , 72.27615 , 76.05667 ],
       [85.05689 , 74.26629 , 78.00687 , 74.886795, 83.13417 ]],
      dtype=float32)>

In [118]:
tf.matmul(tf.transpose(random_A), random_B)

<tf.Tensor: shape=(300, 300), dtype=float32, numpy=
array([[1.317161  , 0.61993605, 1.2612379 , ..., 1.5290778 , 1.0735596 ,
        1.6227092 ],
       [1.0170685 , 0.42642498, 0.8181824 , ..., 1.1469344 , 0.8212255 ,
        1.1739546 ],
       [0.45034647, 0.8037954 , 1.4656199 , ..., 1.105671  , 0.88152766,
        1.481925  ],
       ...,
       [1.3204696 , 1.1634867 , 1.7423928 , ..., 1.8386563 , 1.5207756 ,
        1.5979093 ],
       [0.73207504, 0.90400356, 1.8493464 , ..., 1.3821819 , 0.98218614,
        1.924531  ],
       [1.0814031 , 0.5316744 , 0.7174167 , ..., 1.2942287 , 1.0804075 ,
        1.0476992 ]], dtype=float32)>

## Multiply the two using dotproduct

In [119]:
tf.tensordot(random_A, random_B, axes=0)

<tf.Tensor: shape=(5, 300, 5, 300), dtype=float32, numpy=
array([[[[4.57146466e-01, 3.21966261e-01, 6.18703604e-01, ...,
          4.59927619e-01, 2.20503122e-01, 6.12232029e-01],
         [1.81890175e-01, 7.06553087e-02, 4.13265705e-01, ...,
          2.91221023e-01, 2.02211007e-01, 3.42098713e-01],
         [6.11561956e-03, 2.47750491e-01, 5.43439090e-01, ...,
          3.77380133e-01, 3.26972634e-01, 6.57428086e-01],
         [5.88856936e-01, 5.76324835e-02, 8.08152854e-02, ...,
          6.09873176e-01, 4.82111186e-01, 5.50261259e-01],
         [2.43830979e-01, 6.11405969e-01, 6.41095340e-01, ...,
          4.58627582e-01, 4.74320024e-01, 1.71758875e-01]],

        [[3.03364694e-01, 2.13658452e-01, 4.10574824e-01, ...,
          3.05210292e-01, 1.46326989e-01, 4.06280279e-01],
         [1.20703243e-01, 4.68872190e-02, 2.74245232e-01, ...,
          1.93255737e-01, 1.34188250e-01, 2.27018446e-01],
         [4.05835640e-03, 1.64408475e-01, 3.60628933e-01, ...,
          2.50431389e-0

## Create a tensor with random values between 0 and 1 with shape (224, 224, 3)

In [120]:
random_C = tf.constant(tf.random.uniform(shape=(224,224,3)))
random_C, random_C.shape

(<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
 array([[[0.7413678 , 0.62854624, 0.01738465],
         [0.3431449 , 0.51063764, 0.3777541 ],
         [0.07321596, 0.02137029, 0.2871771 ],
         ...,
         [0.98953485, 0.45382905, 0.2006687 ],
         [0.6295223 , 0.4937899 , 0.01816809],
         [0.95386636, 0.11542463, 0.85691285]],
 
        [[0.78435016, 0.7826872 , 0.87936425],
         [0.24906898, 0.3207239 , 0.10955775],
         [0.543224  , 0.7151396 , 0.40334642],
         ...,
         [0.2445668 , 0.01746976, 0.9036933 ],
         [0.02975535, 0.592268  , 0.9877522 ],
         [0.36701274, 0.33112562, 0.5638567 ]],
 
        [[0.15829337, 0.7288823 , 0.3366307 ],
         [0.70792687, 0.16910625, 0.9429966 ],
         [0.10120225, 0.5919596 , 0.8687303 ],
         ...,
         [0.28134012, 0.10011208, 0.37038183],
         [0.77874243, 0.05421627, 0.4664607 ],
         [0.2549187 , 0.7968637 , 0.83405185]],
 
        ...,
 
        [[0.32922816, 0.06343532

## Find the min and max values of the above

In [121]:
# Minimum value
tf.reduce_min(random_C)

<tf.Tensor: shape=(), dtype=float32, numpy=2.3841858e-07>

In [122]:
# Maximum value
tf.reduce_max(random_C)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9999918>

## Create a tensor with random values of shape (1, 224, 224, 3), then squeeze it to (224, 224, 3)

In [124]:
random_D = tf.constant(tf.random.uniform(shape=(1,224,224,3)))
random_D

<tf.Tensor: shape=(1, 224, 224, 3), dtype=float32, numpy=
array([[[[8.0315602e-01, 4.9777734e-01, 3.7054038e-01],
         [9.1186738e-01, 6.3764203e-01, 1.8209696e-01],
         [6.3791955e-01, 2.7701473e-01, 4.2271137e-02],
         ...,
         [1.0830712e-01, 4.5979273e-01, 2.5716281e-01],
         [8.7138689e-01, 1.8434000e-01, 4.4757760e-01],
         [7.4110627e-02, 9.0852141e-01, 5.3693414e-01]],

        [[5.5596435e-01, 6.8776274e-01, 7.6051474e-02],
         [1.6737962e-01, 7.1785092e-01, 2.7642274e-01],
         [2.6995218e-01, 3.2203627e-01, 8.8224900e-01],
         ...,
         [4.8168826e-01, 5.0150025e-01, 8.6756039e-01],
         [4.1261053e-01, 1.2770486e-01, 5.8186901e-01],
         [2.5495613e-01, 3.9036548e-01, 9.8529553e-01]],

        [[8.0935180e-01, 1.9740558e-01, 3.5899937e-01],
         [1.1216915e-01, 9.1016293e-04, 3.6382091e-01],
         [5.1202202e-01, 3.9188230e-01, 8.8335538e-01],
         ...,
         [2.0133841e-01, 9.1663551e-01, 1.9890130e-01],


In [126]:
tf.squeeze(random_D).shape

TensorShape([224, 224, 3])

## Create a tensor with shape 10, using your own choice of values, then find the index which has the maximum value

In [128]:
my_own_tensor = tf.constant([3, 5, 3, 0, 8, 10, 5, 4, 7, 1])
my_own_tensor

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 3,  5,  3,  0,  8, 10,  5,  4,  7,  1], dtype=int32)>

In [129]:
tf.argmax(my_own_tensor)

<tf.Tensor: shape=(), dtype=int64, numpy=5>

In [130]:
my_own_tensor[tf.argmax(my_own_tensor)]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

## One-hot-encode the above tensor

In [132]:
tf.one_hot(my_own_tensor, depth=tf.size(my_own_tensor))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>